In [1]:
import random as rd

Variant = 4
rd.seed(Variant)


colors = ['COLOR_BLACK', 'COLOR_RED', 'COLOR_GREEN', 'СOLOR_YELLOW', 'COLOR_BLUE', 'COLOR_MAGENTA', 'COLOR_CYAN', 'COLOR_WHITE']
buttons = ['q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', 'a', 's', 'd', 'f', 'g', 'h', 'j','k','l','z','x','c','v','b','n','m']
print("Алгоритм: " , rd.sample([1,2,3,4,5,6,7,8],1))
print("Цвета заполнения: " , rd.sample(colors,2))
print("Кнопки выхода, изменения скорости: " , rd.sample(buttons,2))

Алгоритм:  [4]
Цвета заполнения:  ['COLOR_BLUE', 'COLOR_BLACK']
Кнопки выхода, изменения скорости:  ['b', 'd']


main.asm

In [ ]:
format ELF64

public main

;; Указываем необходимые внешние функции из библиотеки ncurses
extrn initscr
extrn printw
extrn refresh
extrn getch
extrn endwin
extrn exit
extrn stdscr
extrn getmaxx
extrn getmaxy
extrn move
extrn attron
extrn attroff
extrn COLOR_PAIR
extrn init_pair
extrn nodelay

section '.data'

hello db "Press 'b' to exit, 'd' to change speed", 0
color_blue db 4   ; Индекс для синего цвета в ncurses
color_black db 0  ; Индекс для черного цвета в ncurses
speed dd 200000   ; Начальная скорость (интервал в цикле) теперь 4 байта

;; Переменные для координат и направления змейки
x dq 0  ; 8 байт для переменной x (координаты)
y dq 0  ; 8 байт для переменной y (координаты)
direction_x db 1  ; Направление по x (1 - вправо, -1 - влево)
direction_y db 1  ; Направление по y (1 - вниз, -1 - вверх)

section '.text' executable

main:
    ; Инициализация ncurses
    call initscr

    ; Инициализация цветов
    mov rdi, 1      ; создаем цветовую пару
    mov rsi, 4      ; синий цвет
    mov rdx, 0      ; черный цвет
    call init_pair  ; инициализация цветовой пары

    ; Устанавливаем начальную позицию для змейки (центр экрана)
    mov rdi, [stdscr]   ; загружаем экран
    call getmaxx        ; получаем ширину экрана
    mov r8, rax         ; сохраняем ширину в r8
    call getmaxy        ; получаем высоту экрана
    mov r9, rax         ; сохраняем высоту в r9

    ; Устанавливаем начальные координаты змейки (центр экрана)
    mov r8, r8          ; координаты x = ширина экрана
    shr r8, 1           ; делим пополам (центр по x)
    mov [x], r8         ; сохраняем в x
    mov r9, r9          ; координаты y = высота экрана
    shr r9, 1           ; делим пополам (центр по y)
    mov [y], r9         ; сохраняем в y

    ; Устанавливаем nodelay для асинхронного ввода
    mov rdi, [stdscr]
    call nodelay

game_loop:
    ; Отображаем текущую позицию змейки с синим цветом
    mov rdi, [x]   ; x-координата
    mov rsi, [y]   ; y-координата
    call move      ; перемещаем курсор
    mov rdi, color_blue
    call attron     ; включаем синий цвет
    call printw    ; выводим символ (след)
    call attroff    ; выключаем синий цвет

    ; Проверяем нажатие клавиши
    call getch
    cmp al, 'b'     ; если нажата клавиша 'b' - выход
    je end_program
    cmp al, 'd'     ; если нажата клавиша 'd' - изменить скорость
    je change_speed

    ; Изменение координат змейки
    mov al, [direction_x]
    add [x], rax
    mov al, [direction_y]
    add [y], rax

    ; Проверка на достижение границы экрана
    mov rax, [x]
    cmp rax, r8
    jge edge_hit_x
    cmp rax, 0
    jle edge_hit_x

    ; Проверка на достижение границы по Y
    mov rax, [y]
    cmp rax, r9
    jge edge_hit_y
    cmp rax, 0
    jle edge_hit_y

    ; Задержка для изменения скорости
    call delay_speed

    ; Переход в следующий цикл игры
    jmp game_loop

edge_hit_x:
    neg byte [direction_x]  ; меняем направление по X
    jmp game_loop

edge_hit_y:
    neg byte [direction_y]  ; меняем направление по Y
    jmp game_loop

change_speed:
    ; Уменьшаем задержку для увеличения скорости
    sub dword [speed], 10000  ; уменьшаем задержку
    cmp dword [speed], 100000
    jg change_speed_end
    jmp game_loop

change_speed_end:
    jmp game_loop

end_program:
    call endwin       ; завершение работы с ncurses
    call exit         ; выход из программы

delay_speed:
    ; Задержка для регулировки скорости
    mov ecx, [speed]   ; Загружаем значение speed в 32-битный регистр ecx
delay_loop:
    dec ecx
    jnz delay_loop     ; Если ecx != 0, продолжаем цикл
    ret
